<a href="https://colab.research.google.com/github/samipn/clustering_demos/blob/main/dbscan_pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment (d): DBSCAN Clustering using PyCaret

This notebook uses the PyCaret library to perform DBSCAN clustering on a non-convex dataset and evaluates clustering quality.


In [1]:
!pip install pycaret


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pmdarima to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 6.8 

In [21]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_moons
from pycaret.clustering import setup, create_model, plot_model, assign_model, pull

# Create non-convex dataset (two moons)
X, _ = make_moons(n_samples=600, noise=0.07, random_state=42)
df = pd.DataFrame(X, columns=["x1", "x2"])
df.head()


,x1,x2
0,0.465443,0.949830
1,0.688884,-0.444873
2,1.836104,0.123468
3,0.752228,0.707265
4,-0.321856,0.863399


In [22]:
# PyCaret setup for clustering
exp = setup(
    data=df,
    normalize=True,
    session_id=42,
    verbose=True
)


,Description,Value
0,Session id,42
1,Original data shape,"(600, 2)"
2,Transformed data shape,"(600, 2)"
3,Numeric features,2
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,Normalize,True
9,Normalize method,zscore


In [28]:
# Create DBSCAN model & inspect results
dbscan = create_model('dbscan', eps=0.2, min_samples=10)  # Adjusted parameters for better clustering
results = pull()  # get results table
results

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.2283,266.1625,2.1312,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.2283,266.1625,2.1312,0,0,0


In [29]:
plot_model(dbscan, plot='cluster')  # scatter plot of clusters

# Assign cluster labels to original data
dbscan_assigned = assign_model(dbscan)
dbscan_assigned.head()

,x1,x2,Cluster
0,0.465443,0.949830,Cluster 0
1,0.688884,-0.444873,Cluster 1
2,1.836104,0.123468,Cluster 1
3,0.752228,0.707265,Cluster 0
4,-0.321856,0.863398,Cluster 0


In [30]:
from sklearn.metrics import silhouette_score

labels = dbscan_assigned['Cluster'].values
X_cluster = dbscan_assigned[['x1', 'x2']].values

unique_labels = np.unique(labels)
if len(unique_labels) > 1:
    mask = labels != -1  # exclude noise
    if len(np.unique(labels[mask])) > 1:
        sil = silhouette_score(X_cluster[mask], labels[mask])
        print("Silhouette score (excluding noise):", sil)
    else:
        print("Too few non-noise clusters for silhouette.")
else:
    print("Silhouette score cannot be computed (only one cluster).")

Silhouette score (excluding noise): 0.1887532
